In [ ]:
import pandas as pd
import numpy as np

### Explicación de la limpieza de datos

A continuación, se realiza una explicación detallado de la limpieza aplicada a las columnas del DataFrame de juegos:

-	Había nombres duplicados, porque algunas reediciones se llamaban igual, por eso, se utilizaron los link como clave primaria.

-	La puntuación se convirtió a número flotante.

-	A las columnas con listas, como Autores, Categorías, Mecánicas y Juegos relacionados, se cambiaron los NaN por Desconocido.

-	Como los juegos de mesa base no tienen juegos relacionados, porque son los juegos principales, los valores nulos pasaron a Ninguno.

-	A las columnas que no tenían outliers se les aplicó la media.

-	A Duración y Jugadores se le aplicó la mediana, se dividieron en mínimos y máximos y se crearon dos nuevas columnas con el resultado de su media. Se cambiaron los valores nulos, sabiendo que este procedimiento tiende a centrar visualmente los valores al graficar y que no es necesario para el análisis, pero se realizó puesto que la cantidad de valores nulos no era lo suficientemente alta como para afectar considerablemente la visualización gráfica.

-	En el df de precios se creó una nueva columna con el precio medio y se unió con un merge al df de juegos.


Lectura del CSV JUEGOS


In [ ]:
df_juegos = pd.read_csv("juegos_ludonauta.csv")


Renombrar columnas para eliminar las tildes

In [ ]:
df_juegos.rename(columns={'Unnamed: 0' : 'Link',
                          'Fecha publicación' : 'Fecha publicacion',
                          'Categorías' : 'Categorias',
                          'Mecánicas' : 'Mecanicas',
                          'Puntuación' : 'Puntuacion',
                          'Jugadores mínimos' : 'Jugadores minimos',
                          'Jugadores máximos' : 'Jugadores maximos',
                          'Duración' : 'Duracion',
                          'Duración mínima' : 'Duracion minima',
                          'Duración máxima' : 'Duracion maxima',
                          'Edición juegos' : 'Edicion juegos'}, inplace= True)

In [ ]:
df_juegos.info()

In [ ]:
# Visualización de los NaN's

import plotly.express as px

fig = px.imshow(df_juegos.isna(), labels=dict(x="Columnas", y="Filas"), x=df_juegos.columns, y=df_juegos.index)

fig.update_layout(
    title='Visualización de NaNs'
)

fig.show()

In [ ]:
df_juegos.head()

La cantidad de únicos de Link es la misma que de su tamaño total, es decir, cada fila de Link representa un valor único. Link lo utilizaremos de clave primaria con valores únicos.

In [ ]:
df_juegos["Link"].nunique() == df_juegos.shape[0]


Conversión de los string a lista

In [ ]:
columnas = ['Autores', 'Categorias', 'Mecanicas', 'Juegos relacionados']

for columna in columnas:
    df_juegos[columna] = df_juegos[columna].apply(lambda x: eval(x) if not pd.isna(x) else None)

Conversión de la columna Puntuacion a Typefloat

In [ ]:
df_juegos['Puntuacion'] = df_juegos['Puntuacion'].apply(lambda x : float(x.replace(',', '.')))

In [ ]:
df_juegos.info()

In [ ]:
#Esto muestra que los valores nulos de Juegos relacionados se corresponden a los Juego de mesa base
df_juegos["Edicion juegos"].value_counts()

In [ ]:
# Reemplazar valores NaN en la columna Juegos relacionados por "Ninguno", porque no tienen relación con ninguno.
df_juegos['Juegos relacionados'] = df_juegos['Juegos relacionados'].apply(lambda d: d if isinstance(d, list) else ['Ninguno'])

In [ ]:
# Ya no hay valores NaN
nan_juegos_relacionados = df_juegos[df_juegos["Juegos relacionados"].isna()]
nan_juegos_relacionados

In [ ]:
df_juegos[df_juegos["Edicion juegos"] == "Juego de mesa base"]

In [ ]:
edad = df_juegos["Edad"].unique()
edad.sort()
print(edad)

In [ ]:
# No hay valores outliers
df_juegos["Edad"].value_counts()

In [ ]:
# Le aplicamos la media de edad a los valores NaN
media_edad = df_juegos['Edad'].mean()
df_juegos["Edad"].fillna(value= media_edad, inplace=True)

In [ ]:
# Le aplicamos la media de complejidad a los valores NaN
media_complejidad = df_juegos['Complejidad'].mean()
df_juegos["Complejidad"].fillna(value= media_complejidad, inplace=True)

In [ ]:
# Le aplicamos la media de Dep idioma a los valores NaN
media_idioma = df_juegos['Dep idioma'].mean()
df_juegos["Dep idioma"].fillna(value= media_idioma, inplace=True)

In [ ]:
# En duración mínima y máxima hay outliers, por lo que aplicamos mediana
df_juegos["Duracion minima"].value_counts()

In [ ]:
# Aplicamos mediana a duración mínima
mediana_duracion_minima = df_juegos['Duracion minima'].median()
df_juegos["Duracion minima"].fillna(value= mediana_duracion_minima, inplace=True)

In [ ]:
# Aplicamos mediana a duración máxima
mediana_duracion_maxima = df_juegos['Duracion maxima'].median()
df_juegos["Duracion maxima"].fillna(value= mediana_duracion_maxima, inplace=True)

In [ ]:
# Aplicamos la mediana a la duración en una nueva columna: "Mediana duración"
df_juegos['Mediana duracion'] = df_juegos[['Duracion minima', 'Duracion maxima']].median(axis=1)

In [ ]:
# No tiene valores NaN la columna "Mediana duración"
nan_mediana_duracion = df_juegos[df_juegos["Mediana duracion"].isna()]
nan_mediana_duracion

In [ ]:
# Rellenamos los NaN de Duración con la mediana, que está recogida en 'Mediana duración'
df_juegos['Duracion'].fillna(df_juegos['Mediana duracion'], inplace=True)

In [ ]:
# Jugadores vacío se había guardado "-", lo modificamos
df_juegos["Jugadores"].replace("-", np.nan, inplace= True)

In [ ]:
df_juegos["Jugadores minimos"].value_counts()

In [ ]:
# Aplicamos mediana a jugadores mínimos y rellenamos los NaN
mediana_jugadores_minimos = df_juegos["Jugadores minimos"].median()
df_juegos["Jugadores minimos"].fillna(value= mediana_jugadores_minimos, inplace=True)

In [ ]:
# Aplicamos mediana a jugadores máximos, y rellenamos los NaN
mediana_jugadores_maximos = df_juegos["Jugadores maximos"].median()
df_juegos["Jugadores maximos"].fillna(value= mediana_jugadores_maximos, inplace=True)

In [ ]:
# Aplicamos la media a los jugadores en una nueva columna: "Mediana jugadores"
df_juegos["Mediana jugadores"] = df_juegos[["Jugadores minimos", "Jugadores maximos"]].median(axis=1)

In [ ]:
# Rellenamos los NaN de Jugadores con la media, que está recogida en "Mediana jugadores"
df_juegos["Jugadores"].fillna(df_juegos["Mediana jugadores"], inplace=True)

In [ ]:
# Reemplazamos los valores NaN por "Desconocido" en: Autores, Categorías y Mecánicas.
df_juegos['Autores'] = df_juegos['Autores'].apply(lambda d: d if isinstance(d, list) else ['Desconocido'])
df_juegos['Categorias'] = df_juegos['Categorias'].apply(lambda d: d if isinstance(d, list) else ['Desconocido'])
df_juegos['Mecanicas'] = df_juegos['Mecanicas'].apply(lambda d: d if isinstance(d, list) else ['Desconocido'])

In [ ]:
# Reemplazamos en la columna Autores '(No acreditado)' por 'Desconocido'
def cambiar_string(lista):
    return [item.replace('(No acreditado)', 'Desconocido') if isinstance(item, str) else item for item in lista]

df_juegos['Autores'] = df_juegos['Autores'].apply(cambiar_string)

In [ ]:
# Eliminar en la columna Edicion juegos que termine en 'de'
df_juegos['Edicion juegos'] = df_juegos['Edicion juegos'].apply(lambda x: x.replace(' de', ''))

In [ ]:
# No tenemos juegos duplicados, aunque les hayan puesto el mismo nombre a las reediciones
duplicados = df_juegos.duplicated(subset=['Nombre', 'Fecha publicacion', 'Edicion juegos'])
filas_duplicadas = df_juegos[duplicados]
filas_duplicadas

In [ ]:
# Número de Nan's en cada columna
df_juegos.isna().sum()

In [ ]:
df_juegos.columns

In [ ]:
df_juegos.shape

In [ ]:
df_juegos.head()

In [ ]:
duplicados = df_juegos.duplicated(subset=['ID_Producto'])

# Mostrar los registros duplicados
print(df_juegos[duplicados])

In [ ]:
# CSV CON LOS DATOS LIMPIOS DE JUEGOS

df_juegos.to_csv("juegos_ludonauta_limpios.csv", index= False)